In [ ]:
# Initialize libraries
library("methylKit")
library("genomation")
library("clusterProfiler")
library("biomaRt")
library("org.Mm.eg.db")

In [ ]:
# Prepare a list of files to be analyzed
file.list = list('/home/Aleksandra.Gorbunova/met/final_proj/non_CpG_iPSCder_rep1_chr5.bismark.cov.gz',
                 '/home/Aleksandra.Gorbunova/met/final_proj/non_CpG_iPSCder_rep2_chr5.bismark.cov.gz',
                 '/home/Aleksandra.Gorbunova/met/final_proj/non_CpG_iPSCder_rep3_chr5.bismark.cov.gz',
                 '/home/Aleksandra.Gorbunova/met/final_proj/non_CpG_pm_rep1_chr5.gz.bismark.cov.gz',
                 '/home/Aleksandra.Gorbunova/met/final_proj/non_CpG_pm_rep2_chr5.gz.bismark.cov.gz',
                 '/home/Aleksandra.Gorbunova/met/final_proj/non_CpG_pm_rep3_chr5.gz.bismark.cov.gz')

# Read the files into a methylKit object of type methylRawList: myobj
myobj = methRead(file.list,
                 pipeline="bismarkCoverage",
                 sample.id=list("iPSCder_1","iPSCder_2","iPSCder_3","pm_1","pm_2","pm_3"),
                 assembly="hg38",
                 treatment=c(1,1,1,0,0,0),
                 context="CpG",
                 )

In [ ]:
filtered_data <- filterByCoverage(
  myobj,
  lo.count = 10, # minimal covarege per site
  lo.perc = NULL,
  hi.count = 500, # max covarege per site (protection from PCR bias)
  hi.perc = NULL
)

In [ ]:
sapply(filtered_data, nrow)

In [ ]:
# Visualisation of coverage for each sample
pdf("coverage_distribution_nonCpG.pdf", width = 10, height = 6)
par(mfrow = c(2, 3))
for(i in 1:length(filtered_data)){
  getCoverageStats(filtered_data[[i]], plot = TRUE)
}
dev.off()

In [ ]:
#normalisation and visualisation
normalized_data <- normalizeCoverage(filtered_data)
pdf("coverage_distribution_after_norm.pdf", width = 10, height = 6)
par(mfrow = c(2, 3)) # 2 строки, 3 колонки для 6 образцов
for(i in 1:length(normalized_data)){
  getCoverageStats(normalized_data[[i]], plot = TRUE)
}
dev.off()

In [ ]:
meth = unite(filtered_data) 
dim(meth)

Clustering analysis

In [ ]:
clusterSamples(meth, dist="correlation", method="ward", plot=TRUE)

In [ ]:
pc = PCASamples(meth, obj.return=TRUE, adj.lim=c(1,1))

Differential methilation analysis

In [ ]:
myDiff = calculateDiffMeth(meth, test = "Chisq", overdispersion ='MN', mc.cores=1)

In [ ]:
#save in bedgraph format
bedgraph(
  methylObj = myDiff,
  file.name = "myDiff_nonCpG.bedgraph",
  col.name = "meth.diff"  
)

Extract significant differentially methylated sites

In [ ]:
myDiff20 = getMethylDiff(myDiff, difference=20, qvalue=0.1)
#used this parameters due to low amount of results for more strong one

In [ ]:
#save in bedgraph format
bedgraph(
  methylObj = myDiff20,
  file.name = "myDiff20_nonCpG.bedgraph",
  col.name = "meth.diff"  
)

Annotation of differentially methylated sites

In [ ]:
refseq_anot <- readTranscriptFeatures("/home/Aleksandra.Gorbunova/met/final_proj/hg38_for_annotation_bed")

In [ ]:
refseq_df <- as.data.frame(refseq_anot)

In [ ]:
bed_columns <- c("chr", "start", "end", "strand")
myDiff_df <- getData(myDiff)
diff_bed <- myDiff_df[, bed_columns]

In [ ]:
# we couldn't use simple annotation, cause we want to look at distances to 
# nearest feature (gene or promoter)
# diff_bed to GRanges
diff_gr <- GRanges(
  seqnames = diff_bed$chr,
  ranges = IRanges(start = diff_bed$start, end = diff_bed$end),
  strand = diff_bed$strand
)

# exons to genes
genes_df <- refseq_df %>%
  filter(group_name != "promoters") %>%
  group_by(name, seqnames, strand) %>%
  summarise(start = min(start), end = max(end), .groups = "drop")

genes_gr <- GRanges(
  seqnames = genes_df$seqnames,
  ranges = IRanges(start = genes_df$start, end = genes_df$end),
  strand = genes_df$strand,
  gene_id = genes_df$name
)

# promoters agregation 
promoters_df <- refseq_df %>%
  filter(group_name == "promoters")

promoters_gr <- GRanges(
  seqnames = promoters_df$seqnames,
  ranges = IRanges(start = promoters_df$start, end = promoters_df$end),
  strand = promoters_df$strand,
  promoter_id = promoters_df$name 
)

# for each position looking for nearest promoter and gene
nearest_promoter <- nearest(diff_gr, promoters_gr, ignore.strand = TRUE)
nearest_gene <- nearest(diff_gr, genes_gr, ignore.strand = TRUE)

# calculating distances
dist_to_promoter <- rep(NA_integer_, length(diff_gr))
dist_to_gene <- rep(NA_integer_, length(diff_gr))

hits_promoter <- which(!is.na(nearest_promoter))
dist_to_promoter[hits_promoter] <- distance(diff_gr[hits_promoter], promoters_gr[nearest_promoter[hits_promoter]])

hits_gene <- which(!is.na(nearest_gene))
dist_to_gene[hits_gene] <- distance(diff_gr[hits_gene], genes_gr[nearest_gene[hits_gene]])

# choose the nearest
nearest_element <- character(length(diff_gr))
nearest_element_id <- character(length(diff_gr))
nearest_distance <- numeric(length(diff_gr))

for(i in seq_along(diff_gr)) {
  p_dist <- dist_to_promoter[i]
  g_dist <- dist_to_gene[i]
  
  if (is.na(p_dist) && is.na(g_dist)) {
    nearest_element[i] <- NA
    nearest_element_id[i] <- NA
    nearest_distance[i] <- NA
  } else if (!is.na(p_dist) && (is.na(g_dist) || p_dist <= g_dist)) {
    nearest_element[i] <- "promoter"
    nearest_element_id[i] <- promoters_gr$promoter_id[nearest_promoter[i]]
    nearest_distance[i] <- p_dist
  } else {
    nearest_element[i] <- "gene"
    nearest_element_id[i] <- genes_gr$gene_id[nearest_gene[i]]
    nearest_distance[i] <- g_dist
  }
}

# for each promoter seeking for the gene (start > end)

linked_gene_for_promoter <- rep(NA_character_, length(diff_gr))

promoter_idx <- which(nearest_element == "promoter")
for (i in promoter_idx) {
  prom_chr <- as.character(seqnames(promoters_gr[nearest_promoter[i]]))
  prom_end <- end(promoters_gr[nearest_promoter[i]])
  
  candidate_genes <- genes_gr[seqnames(genes_gr) == prom_chr & start(genes_gr) > prom_end]
  
  if (length(candidate_genes) > 0) {
    distances_downstream <- start(candidate_genes) - prom_end
    linked_gene_for_promoter[i] <- candidate_genes$gene_id[which.min(distances_downstream)]
  }
}

diff_bed$nearest_element <- nearest_element
diff_bed$nearest_id <- nearest_element_id
diff_bed$distance_to_nearest <- nearest_distance
diff_bed$linked_gene_for_promoter <- linked_gene_for_promoter


In [ ]:
#save results in csv
write.csv(diff_bed, file = "diff_bed_with_distances_nonCpG.csv", row.names = FALSE)


In [ ]:
#visualisation
library(dplyr)
library(ggplot2)

plot_df <- diff_bed %>%
  filter(!is.na(distance_to_nearest)) %>%
  mutate(feature_type = ifelse(nearest_element == "gene", "Gene",
                               ifelse(nearest_element == "promoter", "Promoter", NA))) %>%
  filter(!is.na(feature_type))

In [ ]:
ggplot(plot_df, aes(x = distance_to_nearest, fill = feature_type)) +
  geom_histogram(position = "identity", alpha = 0.5, bins = 50) +
  scale_x_log10(name = "Distance to nearest feature (bp, log scale)") +
  scale_y_continuous(name = "Count") +
  labs(title = "Distribution of distances to nearest genes and promoters") +
  theme_minimal() +
  theme(legend.title = element_blank())

In [ ]:
#repeate for 20% diff
myDiff_df_20 <- getData(myDiff25)
diff_bed_20 <- myDiff_df_20[, bed_columns]
diff_bed_20

In [ ]:
diff_gr <- GRanges(
  seqnames = diff_bed_20$chr,
  ranges = IRanges(start = diff_bed_20$start, end = diff_bed_20$end),
  strand = diff_bed_20$strand
)

genes_df <- refseq_df %>%
  filter(group_name != "promoters") %>%
  group_by(name, seqnames, strand) %>%
  summarise(start = min(start), end = max(end), .groups = "drop")

genes_gr <- GRanges(
  seqnames = genes_df$seqnames,
  ranges = IRanges(start = genes_df$start, end = genes_df$end),
  strand = genes_df$strand,
  gene_id = genes_df$name
)

promoters_df <- refseq_df %>%
  filter(group_name == "promoters")

promoters_gr <- GRanges(
  seqnames = promoters_df$seqnames,
  ranges = IRanges(start = promoters_df$start, end = promoters_df$end),
  strand = promoters_df$strand,
  promoter_id = promoters_df$name  
)

nearest_promoter <- nearest(diff_gr, promoters_gr, ignore.strand = TRUE)
nearest_gene <- nearest(diff_gr, genes_gr, ignore.strand = TRUE)

dist_to_promoter <- rep(NA_integer_, length(diff_gr))
dist_to_gene <- rep(NA_integer_, length(diff_gr))

hits_promoter <- which(!is.na(nearest_promoter))
dist_to_promoter[hits_promoter] <- distance(diff_gr[hits_promoter], promoters_gr[nearest_promoter[hits_promoter]])

hits_gene <- which(!is.na(nearest_gene))
dist_to_gene[hits_gene] <- distance(diff_gr[hits_gene], genes_gr[nearest_gene[hits_gene]])

nearest_element <- character(length(diff_gr))
nearest_element_id <- character(length(diff_gr))
nearest_distance <- numeric(length(diff_gr))

for(i in seq_along(diff_gr)) {
  p_dist <- dist_to_promoter[i]
  g_dist <- dist_to_gene[i]
  
  if (is.na(p_dist) && is.na(g_dist)) {
    nearest_element[i] <- NA
    nearest_element_id[i] <- NA
    nearest_distance[i] <- NA
  } else if (!is.na(p_dist) && (is.na(g_dist) || p_dist <= g_dist)) {
    nearest_element[i] <- "promoter"
    nearest_element_id[i] <- promoters_gr$promoter_id[nearest_promoter[i]]
    nearest_distance[i] <- p_dist
  } else {
    nearest_element[i] <- "gene"
    nearest_element_id[i] <- genes_gr$gene_id[nearest_gene[i]]
    nearest_distance[i] <- g_dist
  }
}

linked_gene_for_promoter <- rep(NA_character_, length(diff_gr))

promoter_idx <- which(nearest_element == "promoter")
for (i in promoter_idx) {
  prom_chr <- as.character(seqnames(promoters_gr[nearest_promoter[i]]))
  prom_end <- end(promoters_gr[nearest_promoter[i]])
  
  candidate_genes <- genes_gr[seqnames(genes_gr) == prom_chr & start(genes_gr) > prom_end]
  
  if (length(candidate_genes) > 0) {
    distances_downstream <- start(candidate_genes) - prom_end
    linked_gene_for_promoter[i] <- candidate_genes$gene_id[which.min(distances_downstream)]
  }
}

diff_bed_20$nearest_element <- nearest_element
diff_bed_20$nearest_id <- nearest_element_id
diff_bed_20$distance_to_nearest <- nearest_distance
diff_bed_20$linked_gene_for_promoter <- linked_gene_for_promoter


In [ ]:
write.csv(diff_bed_20, file = "diff_bed_with_distances_20_nonCpG.csv", row.names = FALSE)

In [ ]:
plot_df <- diff_bed_20 %>%
  filter(!is.na(distance_to_nearest)) %>%
  mutate(feature_type = ifelse(nearest_element == "gene", "Gene",
                               ifelse(nearest_element == "promoter", "Promoter", NA))) %>%
  filter(!is.na(feature_type))

ggplot(plot_df, aes(x = distance_to_nearest, fill = feature_type)) +
  geom_histogram(position = "identity", alpha = 0.5, bins = 50) +
  scale_x_log10(name = "Distance to nearest feature (bp, log scale)") +
  scale_y_continuous(name = "Count") +
  labs(title = "Distribution of distances to nearest genes and promoters") +
  theme_minimal() +
  theme(legend.title = element_blank())

We also try to perform GO enrichment analysis

In [ ]:
#firstly we need to transform ENST to ENSG
library(biomaRt)

ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
# remove version of transcript from nearest_id
diff_bed_20$nearest_id_clean <- sub("\\.\\d+$", "", diff_bed_20$nearest_id)

# and also from linked_gene_for_promoter
diff_bed_20$linked_gene_clean <- sub("\\.\\d+$", "", diff_bed_20$linked_gene_for_promoter)


# collect all unique ENST from diff_bed
enst_ids <- unique(c(diff_bed_20$nearest_id_clean, diff_bed_20$linked_gene_clean))
enst_ids <- enst_ids[!is.na(enst_ids) & enst_ids != "." & enst_ids != ""]

# mapping ENST -> ENSG
mapping <- getBM(
  attributes = c("ensembl_transcript_id", "ensembl_gene_id"),
  filters = "ensembl_transcript_id",
  values = enst_ids,
  mart = ensembl
)

enst2ensg <- setNames(mapping$ensembl_gene_id, mapping$ensembl_transcript_id)


In [ ]:
diff_bed_20$nearest_gene_ensg <- enst2ensg[diff_bed_20$nearest_id_clean]
diff_bed_20$linked_gene_ensg <- enst2ensg[diff_bed_20$linked_gene_clean]
new_df <- diff_bed_20[, c("chr", "start", "end", "strand")]

new_df$gene <- ifelse(!is.na(diff_bed_20$nearest_gene_ensg),
                      diff_bed_20$nearest_gene_ensg,
                      diff_bed_20$linked_gene_ensg)

diff_bed_20

In [ ]:
gene_diff_new_df_x1 = table(new_df$gene)
gene_diff_new_df_x1 = names(gene_diff_new_df_x1[gene_diff_new_df_x1>=1])

and also repate it for all dif met results

In [ ]:
gene_Meth = table(new_df$gene)
gene_Meth = names(gene_Meth[gene_Meth>=1])

Enrichment analysis

ego <- enrichGO(gene          = gene_diff_new_df_x1,
                universe      = gene_Meth,
                OrgDb         = org.Mm.eg.db,
                keyType       = 'ENSEMBL',
                ont           = 'all',
                pAdjustMethod = 'BH',
                pvalueCutoff  = 0.1,
                qvalueCutoff  = 0.1)

but "ego" always was null